## In this notebook, we will explore comment with high likes.
-----------------
We assume that comment with high likes means that other user also agree with them, giving that opinion more weights.

In [3]:
import pandas as pd
import numpy as np
from pythainlp.tokenize import word_tokenize
import matplotlib.pyplot as plt
import matplotlib as mpl


mpl.rc('font',family='JasmineUPC')

import plotly 
plotly.tools.set_credentials_file(username='TeerapatChaiwachirasak', api_key='RMqZv6z9pSmGeRafgNnS')
import plotly.plotly as py
import plotly.graph_objs as go

In [4]:
#Test reading from .tsv
commentDf = pd.read_csv('../dataset/facebook_comment.tsv',sep ='\t')
commentDf['message'] = np.nan_to_num(commentDf['message'])
commentDf = commentDf[['commenter_name','created_time','like_count','message','service']]
commentDf['created_date'] = pd.to_datetime(commentDf['created_time'], format='%Y-%m-%d %H:%M:%S.%f')
commentDf['created_date_format'] = commentDf['created_date'].apply(lambda x: x.replace(hour=0, minute=0, second=0, microsecond=0))
commentDf['service'][commentDf['service']==True] = 'TRUE'
commentDf.head()

C:\Program Files\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning:

Columns (0,7) have mixed types. Specify dtype option on import or set low_memory=False.

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,commenter_name,created_time,like_count,message,service,created_date,created_date_format
0,Nui Worabut,2016-06-01 12:53:00,1.0,ใช้โปรเสริม net non..9 gb ใช้หมดลดความเร็วไม่...,DTAC,2016-06-01 12:53:00,2016-06-01
1,Chandratithaya Je-u,2016-06-02 17:17:32,0.0,ใช้ดีแทค แล้วปวดใจจริงๆ สัญญานขาดหาย อินเตอร์เ...,DTAC,2016-06-02 17:17:32,2016-06-02
2,Hataichanok Maklamai,2016-06-02 07:51:45,0.0,ส่งขอมูลไปทาง ib แล้วนะคะ รบกวนตอบด้วยคะ ส่งไป...,DTAC,2016-06-02 07:51:45,2016-06-02
3,Nattapong Wongsa,2016-06-02 15:43:53,1.0,อะไรนี้...ไม่เปิดให้เติมเกมส์ใช้ปะ จะได้เลิกใช้,DTAC,2016-06-02 15:43:53,2016-06-02
4,Jitsak Sisawad,2016-06-02 15:09:06,0.0,ห่วยแตกไม่เคยแจ้งข่าวสารให้ลูก อยู่ๆก้มาเก็บค่...,DTAC,2016-06-02 15:09:06,2016-06-02


In [27]:
commentDf['like_count'].value_counts().sort_values().head(10)

81.0     1
536.0    1
471.0    1
466.0    1
421.0    1
420.0    1
376.0    1
375.0    1
372.0    1
367.0    1
Name: like_count, dtype: int64

### Let's look at comment with more than 200 likes.

In [28]:
commentDf[commentDf['like_count']>200]

,commenter_name,created_time,like_count,message,service,created_date,created_date_format
4945,Meenniie Muniin,2016-06-27 07:59:52,219.0,เหอะๆ บอกเลย ทำโปรแต่ไม่มีของ จะทำเพื่อ แถมเจอ...,DTAC,2016-06-27 07:59:52,2016-06-27
7386,นางมารร้าย รินรดา,2016-07-07 09:31:27,397.0,ไม่จริงหรอกพึ่งไปซื้อมาที่ศูนย์ดีแทคพระราม2น่า...,DTAC,2016-07-07 09:31:27,2016-07-07
9828,Truadkhunthod Piya Suphot,2016-07-15 02:21:38,466.0,ผมทำงานอยู่ไต้หวันสมัครเน็ตความเร็วไม่อันผมใช้...,DTAC,2016-07-15 02:21:38,2016-07-15
9829,Laddawan Subsai,2016-07-15 05:25:57,227.0,เดือน599ใช้เน็ตได้16. GBได้ที่ไหนไช้ได้แค่8GBแ...,DTAC,2016-07-15 05:25:57,2016-07-15
10785,Ake Krungkao,2016-07-27 10:10:10,420.0,ใครคิดจะย้ายมาdtac\nหยุดความคิดไว้ตรงนั้นเลย ส...,DTAC,2016-07-27 10:10:10,2016-07-27
19252,TaTa Minnie,2016-10-23 15:43:48,244.0,เป็นลูกค้า dtac มา 13 ปี วันนนี้เดินเข้าไปซื้อ...,DTAC,2016-10-23 15:43:48,2016-10-23
23082,Uma Phiewchan,2016-11-18 05:55:39,800.0,ทำไมคุณไม่คิดที่จะรักษาลูกค้าเก่าไว้บ้าง? เคยอ...,DTAC,2016-11-18 05:55:39,2016-11-18
23083,ปอสี่ ประชาไท,2016-11-18 08:28:48,246.0,ผมลูกค้าเก่า ใช้มาเป็น 20 ปี จะจองเครื่องก็ไม่...,DTAC,2016-11-18 08:28:48,2016-11-18
24460,LookPla Issareekorn,2016-11-24 06:30:12,375.0,พัฒนาสัญญาณบ้างอย่าดีโอ้อวดโฆษณา หมุนติ้วๆๆๆ ห...,DTAC,2016-11-24 06:30:12,2016-11-24
26386,Tapthong Boonsirm,2016-12-01 11:43:55,352.0,พยายามโทรหา call center 1678มา 2 วันละครับ ยั...,DTAC,2016-12-01 11:43:55,2016-12-01


### Let's explore these comment more in details

In [43]:
def analyzeComment(commentNum):
    print("Comment from %s with %d likes"%(commentDf.iloc[commentNum]['commenter_name'],commentDf.iloc[commentNum]['like_count']))
    print(commentDf.iloc[commentNum].message)

In [44]:
analyzeComment(19252)

Comment from TaTa Minnie with 244 likes
เป็นลูกค้า dtac มา 13 ปี วันนนี้เดินเข้าไปซื้อไอโฟน ที่ shop Dtac mbk พนักงานแจ้งว่าซื้อได้ทุกสี ยกเว้นสีดำ ให้สิทธิ์ เฉพาะเปิดเบอร์ใหม่ งง นิดๆ ค่ะ ลค. เก่าไม่ให้ค่ะ และพอเดินไปถาม true และ ais ให้ลูกค้าซื้อได้ทุกสีไม่มีการแยกลูกค้าเก่าใหม่ นอกจาก ย้ายค่ายมามีส่วนลดเพิ่มให้ โมโห เลยค่ะ พอกันที เป็นลูกค้ามา สิบกว่าปี สัญญาณ ก็ไม่ดีไม่เคยคิดจะย้ายค่าย ตอนนี้  เดี๋ยวจะรีบไปย้ายไปค่ายอื่นเลย 👎👎👎👎👎👎👎👎 พนักงานก็ยืนคุยกัน นั่งกดโทรศัพท์บ้าง แย่มากๆ จะถามอะไรก็ ต้องรอเค้าคุยกันเสร็จก่อน


In [45]:
analyzeComment(29597)

Comment from พี่พีท ราศีเมษ with 397 likes
ลื่นนนพร่อง!!! หมุนๆๆๆติ้วยังกะเครื่องซักผ้า ดูยูทูป ก้ขึ้นแตะเพื่อลองใหม่ผมก้แตะๆๆๆ จนมันขึ้นว่าแตะอีกทีกุต่อยนะ น้ำตาแทบไหลไม่มีใครช่วยได้ ไม่ต้องติดต่อใครทั้งนั้นครับ @ วังสมบูรณ์ คลองหาด สระแก้ว


In [46]:
analyzeComment(33870)

Comment from Spk Adul Nakub with 154 likes
ลื่นมากเลยครับ อยากบอกต่อ เวลาดูคลิปน่ะ มีดาวหมุนอยู่กลางด้วยคัฟ ผมนิร้องโห..เลยอ่ะ


In [47]:
analyzeComment(10785)

Comment from Ake Krungkao with 420 likes
ใครคิดจะย้ายมาdtac
หยุดความคิดไว้ตรงนั้นเลย สัณญานแม่งแย่มาก
ตอนแรกอยู่ทรูว่าแย่จนต้องย้ายค่ายเผื่อจะดีขึ้น
แต่ที่ไหนได้ ไม่น่าย้ายเลย
ห่วยขั้นเทพ คิดว่าdtacจะปรับปรุงแล้วซะอีก ห่วยเหมือนเดิม ห่วยๆๆๆๆๆๆ


In [48]:
analyzeComment(38089)

Comment from Parichat Jinakun with 1293 likes
โอ้ยอีดอกด่ากันอยู่นั่นแหละ กูนี่ใช้ทุกค่ายค่ะ ไม่มีค่ายไหนครอบคลุมการใช้งานทั่วประเทศหรอกค่ะ บางพื้นที่ดีแทคมีเอไอเอสดับ บางพื้นที่เอไอเอสมี ดีแทคทรูดับ อยากครอบคลุมก็ซื้อมาใช้3ซิมสิจบ เวลาตั้งเสาสัญญาณคิดว่าใช้เงิน 10 20 บาทหรอแต่ละค่ายน่ะ ไปเปิดบริษัทใช้สัญญาณเองมั้ย เข้าไปดูเพจเครือข่ายไหนก็โดนด่าหมด ไม่รู้ต้องการอะไรกัน #ไปซื้อใช้เลยค่ะสามค่าย #เลิฟอั้ม


### Most comment are either berate, satire, or funny
---